# SHP2CSV algorithm
## Importing libraries

In [16]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [17]:
from shapely.geometry import shape
from shapely.ops import unary_union
import fiona
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from IPython.core.debugger import Tracer
from shutil import copyfile
import pandas as pd

## Importing and reading shp file

## Converting `geoms` into a planar graph

In [18]:
file_info = fiona.open("shp/Rutas_Transmilenio/Rutas_Transmilenio.shp")

## Creating a Graph object in order to print it
this part of the code is not needed, it is just for having an idea of the graph that is going to be exported

## Plotting the graph

## Creating sets

In [19]:
y = set()
x = set()
lengths = set()

nodes = set()
edges = set()
edges_prime = set()

nodes_norm = list()
edges_norm = set()

## Creating a list of nodes

In [20]:
for line in file_info:
    for node in line['geometry']['coordinates']:
        nodes.add((round(node[0],3),round(node[1],3)))
        x.add(round(node[0],3))
        y.add(round(node[1],3))

nodes = list(nodes)

## Normalizing the nodes coordinates

In [21]:
# Getting the max and min for x and y axis
x_max = max(x)
x_min = min(x)
y_max = max(y)
y_min = min(y)

In [22]:
df3 = pd.DataFrame(data=[[x_min,x_max,y_min,y_max]], columns=['x_min','x_max','y_min','y_max'])
df3.to_csv('../../distributions/boundaries.csv', index=False)

In [23]:
# Normalizing

for node in nodes:
    node_norm = (nodes.index(node), [(node[0]-x_min)/(x_max-x_min),(node[1]-y_min)/(y_max-y_min)])
    nodes_norm.append(node_norm)
    
nodes_norm = [(item[0], [round(item[1][0],3), round(item[1][1],3)]) for item in nodes_norm]

## Creating a list of edges

In [24]:
for line in file_info:
    last_coord = None
    for coord in line['geometry']['coordinates']:
        if last_coord != None:
            length = ((coord[0]-last_coord[0])**2+(coord[1]-last_coord[1])**2)**0.5
            edge = ((nodes.index((round(last_coord[0],3),round(last_coord[1],3))),nodes.index((round(coord[0],3),round(coord[1],3)))),length,line['properties']['Id'])
            edges.add(edge)
        last_coord = coord
        
    
edges = [edge for edge in edges if edge[0][1] != edge[0][0]]


## Normalizing the cost of the edges

In [25]:
length_max = max(list(zip(*edges))[1])
length_min = min(list(zip(*edges))[1])

# normalizing length
edges_norm = [(edges.index(edge),edge[0],(edge[1]-length_min)/(length_max-length_min),edge[2]) for edge in edges]

## Exporting results to csv

In [26]:
# Writing csv
import csv

# vertices
with open("vertices.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in nodes_norm:
        writer.writerow([val[0],val[1][0],val[1][1]])
        
# edges
with open("edges.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in edges_norm:
        writer.writerow([val[0], val[1][0], val[1][1], val[2], val[3]])

copyfile('edges.csv', '../../streets/edges.csv');
copyfile('vertices.csv', '../../streets/vertices.csv');

In [27]:
coords = []

for coord in list(zip(*edges))[0]:
    coord1 = (nodes[coord[0]], nodes[coord[1]])
    coords.append(coord1)

In [28]:
coords

[((-74.125, 4.571), (-74.125, 4.564)),
 ((-74.11, 4.628), (-74.104, 4.625)),
 ((-74.108, 4.706), (-74.104, 4.704)),
 ((-74.12, 4.629), (-74.11, 4.628)),
 ((-74.074, 4.606), (-74.067, 4.605)),
 ((-74.071, 4.671), (-74.075, 4.666)),
 ((-74.082, 4.605), (-74.08, 4.603)),
 ((-74.07, 4.622), (-74.073, 4.617)),
 ((-74.11, 4.71), (-74.121, 4.68)),
 ((-74.135, 4.63), (-74.129, 4.629)),
 ((-74.072, 4.61), (-74.074, 4.606)),
 ((-74.084, 4.691), (-74.08, 4.685)),
 ((-74.173, 4.629), (-74.165, 4.633)),
 ((-74.159, 4.638), (-74.153, 4.635)),
 ((-74.121, 4.68), (-74.173, 4.629)),
 ((-74.05, 4.732), (-74.051, 4.727)),
 ((-74.054, 4.699), (-74.056, 4.693)),
 ((-74.07, 4.7), (-74.067, 4.693)),
 ((-74.084, 4.59), (-74.087, 4.586)),
 ((-74.077, 4.622), (-74.072, 4.617)),
 ((-74.136, 4.569), (-74.14, 4.571)),
 ((-74.081, 4.627), (-74.077, 4.622)),
 ((-74.081, 4.627), (-74.084, 4.623)),
 ((-74.102, 4.602), (-74.11, 4.598)),
 ((-74.087, 4.694), (-74.084, 4.691)),
 ((-74.09, 4.582), (-74.094, 4.577)),
 ((-74